### Adding Libraries

In [1]:
import pandas as pd
import os

### Classifying Data

In [8]:
# load CSV file
data = pd.read_csv('../data/dataset.csv', skiprows=1)

# create destination directory
root_save_path = '../data/train/'
if not os.path.exists(root_save_path):
    os.makedirs(root_save_path)

# group images by lable
grouped = data.groupby(data.columns[0])

# extract and save data by in defferent directories
for label, group in grouped:
    label_save_path = os.path.join(root_save_path, str(label))
    if not os.path.exists(label_save_path):
        os.makedirs(label_save_path)
    csv_file_path = os.path.join(label_save_path, f'label_{label}.csv')
    group.to_csv(csv_file_path, header=False, index=False)


### Make Images

In [10]:
import math
import numpy as np
import base64
from PIL import Image
import io

def load_data(filepath):
    """Load CSV data into a DataFrame."""
    return pd.read_csv(filepath)

# load CSV data
data = pd.read_csv('../data/dataset.csv', skiprows=1)

save_path = '../data/image_files/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# read dataset.csv data {lable, pixcels}
for index, row in data.iterrows():
    label = row.iloc[0]
    pixels = row.iloc[1].split(' ')
    pixels = list(map(int, pixels))
    
    # calculate image real sizes
    image_size = int(math.sqrt(len(pixels)))
    image_array = np.array(pixels, dtype=np.uint8).reshape((image_size, image_size))
    
    # creat image and save
    image = Image.fromarray(image_array)
    image.save(os.path.join(save_path, f'label_{label}_index_{index}.png'))

### Detecting Faces

In [ ]:
import cv2

# مسیر پوشه‌ای که تصاویر در آن ذخیره شده‌اند
image_folder_path = 'path_to_your_images/'

# بارگذاری مدل پیش‌آموزش دیده برای تشخیص چهره
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# تابعی برای تشخیص وضوح چهره در تصویر
def detect_face_and_clarity(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # تشخیص چهره‌ها
    faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)
    
    if len(faces) == 0:
        return 0  # اگر چهره‌ای تشخیص داده نشد، وضوح را 0 درصد در نظر می‌گیریم
    
    # محاسبه مساحت کل تصویر
    total_area = image.shape[0] * image.shape[1]
    
    # محاسبه مساحت کل چهره‌های تشخیص داده شده
    face_area = sum((x2 - x) * (y2 - y) for x, y, x2, y2 in faces)
    
    # محاسبه درصد وضوح چهره بر اساس نسبت مساحت چهره به کل تصویر
    clarity_percentage = (face_area / total_area) * 100
    
    return clarity_percentage

# برای هر تصویر در پوشه، تشخیص وضوح چهره را انجام دهید و یک دیکشنری بسازید
clarity_dict = {}
for image_name in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_name)
    clarity_percentage = detect_face_and_clarity(image_path)
    clarity_dict[image_name] = clarity_percentage

# نمایش دیکشنری وضوح چهره
for image_name, clarity in clarity_dict.items():
    print(f"Image: {image_name}, Clarity: {clarity:.2f}%")
